In [400]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import math
from sklearn.neighbors import LocalOutlierFactor

In [401]:
population = pd.read_csv(r"C:\Users\sathv\OneDrive\Desktop\RDM\adult.data.csv")

In [402]:
sample_1 = population

In [403]:
X= sample_1[["age","workclass","education-num","marital-status","occupation","relationship","race","sex","hoursperweek","native-country"]]
y = sample_1["class"]

In [404]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state = 42)

In [405]:
X_train

,age,workclass,education-num,marital-status,occupation,relationship,race,sex,hoursperweek,native-country
5514,33,Local-gov,13,Never-married,Prof-specialty,Not-in-family,White,Female,50,United-States
19777,36,Private,11,Married-civ-spouse,Exec-managerial,Husband,White,Male,50,United-States
10781,58,Self-emp-not-inc,5,Separated,Craft-repair,Not-in-family,White,Male,40,United-States
32240,21,Private,11,Married-civ-spouse,Farming-fishing,Husband,White,Male,46,United-States
9876,27,Private,10,Divorced,Other-service,Not-in-family,White,Male,40,United-States
...,...,...,...,...,...,...,...,...,...,...
29802,47,Private,13,Married-civ-spouse,Craft-repair,Husband,White,Male,40,United-States
5390,31,Private,8,Divorced,Other-service,Unmarried,White,Female,21,United-States
860,18,Private,7,Never-married,Adm-clerical,Own-child,White,Female,20,United-States
15795,50,Self-emp-not-inc,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,84,United-States


In [406]:
y_train

5514       >50K
19777      >50K
10781     <=50K
32240     <=50K
9876      <=50K
          ...  
29802     <=50K
5390      <=50K
860       <=50K
15795     <=50K
23654     <=50K
Name: class, Length: 26048, dtype: object

In [407]:
def detect_outlier(data,col):
    only_na = []
    new_data = data.copy()
    clf = LocalOutlierFactor(n_neighbors=30)
    normalized_df = (new_data[col]- new_data[col].mean())/new_data[col].std()
    clf.fit_predict(normalized_df.values.reshape(-1,1))
    results = clf.negative_outlier_factor_
    new_data["LOF"] = results.tolist()
    new_data = new_data.query("LOF < -1.5")
    for i in new_data.index:
        only_na.append(i)
    return only_na

In [408]:
def impute(data,col,index):
    new_data = data.copy()
    mode = new_data[col].mode()
    for i in index:
        new_data.at[i, col]=mode
    return new_data
        

In [409]:
def drop_rows(x,only_na):
    new_data = x.copy()
    for i in only_na:
        new_data.drop(i,inplace=True)

    return new_data

In [410]:
index = detect_outlier(X_train,"age")

In [411]:
X_train = impute(X_train,"age",index)

In [412]:
index = detect_outlier(X_train,"hoursperweek")

In [413]:
X_train = impute(X_train,"hoursperweek",index)

In [414]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()

In [415]:
def label_encoding(data,cat_cols):
    if cat_cols is None:
        data= label_encoder.fit_transform(data)
        return data
        
    else:
        for i in range(len(cat_cols)):
            data[cat_cols[i]]= label_encoder.fit_transform(data[cat_cols[i]])
        

In [416]:
cols = X_train.select_dtypes([np.object]).columns

C:\Users\sathv\AppData\Local\Temp\ipykernel_1768\2170981920.py:1: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  cols = X_train.select_dtypes([np.object]).columns


In [417]:
cols

Index(['workclass', 'marital-status', 'occupation', 'relationship', 'race',
       'sex', 'native-country'],
      dtype='object')

In [418]:
label_encoding(X_train,cols)

In [419]:
X_train

,age,workclass,education-num,marital-status,occupation,relationship,race,sex,hoursperweek,native-country
5514,33,2,13,4,10,1,4,0,50,39
19777,36,4,11,2,4,0,4,1,50,39
10781,58,6,5,5,3,1,4,1,40,39
32240,21,4,11,2,5,0,4,1,46,39
9876,27,4,10,0,8,1,4,1,40,39
...,...,...,...,...,...,...,...,...,...,...
29802,47,4,13,2,3,0,4,1,40,39
5390,31,4,8,0,8,4,4,0,40,39
860,18,4,7,4,1,3,4,0,20,39
15795,50,6,9,2,5,0,4,1,84,39


In [420]:
# col = None
y_train = label_encoding(y_train, None)

In [421]:
y_train

array([1, 1, 0, ..., 0, 0, 0])

In [422]:
label_encoding(X_test, cols)

In [423]:
y_test= label_encoding(y_test, None)

In [424]:
y_test

array([0, 0, 1, ..., 1, 0, 0])

In [425]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
logisticRegr = LogisticRegression()

In [426]:
def logistic_regression_train(logisticRegr,x,y):
    logisticRegr.fit(x,y)
def logistic_regression_test(logisticRegr,x):
    y_pred = logisticRegr.predict_proba(x)
    return y_pred
def logistic_regression_pred_test(logisticRegr,x):
    y_pred_test = logisticRegr.predict(x)
    return y_pred_test

def average_diff(y_pred, X_test):
    column_i = y_pred.T[0]
    X_test_copy = X_test.copy()
    X_test_copy['preds'] = column_i
    Female = X_test_copy["preds"][X_test_copy["sex"] == 0]
    female_average = sum(Female)/len(Female)
    male = X_test_copy["preds"][X_test_copy["sex"] == 1]
    male_average = sum(male)/len(male)
    diff = (female_average - male_average)
    return diff 

def accuracy(y_test, y_pred_final):
#     y_pred_final = np.argmax(y_pred, axis=1)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred_final).ravel()
    true_positive  = tn
    false_positive = fn
    true_negative  = tp
    false_negative = fp
    accuracy = (true_positive + true_negative) / (true_negative + true_positive + false_positive + false_negative )
    return(accuracy)
    
    

In [427]:
model = logistic_regression_train(logisticRegr, X_train, y_train)

C:\Users\sathv\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [428]:
y_pred = logistic_regression_test(logisticRegr, X_test)

In [429]:
y_pred

array([[0.89548417, 0.10451583],
       [0.91928258, 0.08071742],
       [0.53420644, 0.46579356],
       ...,
       [0.24316377, 0.75683623],
       [0.90430599, 0.09569401],
       [0.95870083, 0.04129917]])

In [430]:
dirty_diff = average_diff(y_pred, X_test)

In [431]:
dirty_diff

0.20728795801520483

In [432]:
dirty_test_score = logisticRegr.score(X_test, y_test)

In [433]:
dirty_test_score

0.7997850452940273

# Clean test data

In [434]:
X_clean_train, X_clean_test, y_clean_train, y_clean_test = train_test_split(X, y, test_size=0.20, random_state = 42)

In [435]:
index= detect_outlier(X_clean_train,"age")

In [436]:
X_clean_train = impute(X_clean_train,"age",index)

In [437]:
index = detect_outlier(X_clean_train,"hoursperweek")

In [438]:
X_clean_train = impute(X_clean_train,"hoursperweek",index)

In [439]:
index = detect_outlier(X_clean_test,"age")

In [440]:
X_clean_test = impute(X_clean_test,"age",index)

In [441]:
index = detect_outlier(X_clean_test,"hoursperweek")

In [442]:
X_clean_test = impute(X_clean_test,"hoursperweek",index)

In [443]:
label_encoding(X_clean_train,cols)

In [444]:
y_clean_train = label_encoding(y_clean_train, None)

In [445]:
label_encoding(X_clean_test,cols)

In [446]:
y_clean_test= label_encoding(y_clean_test, None)

In [447]:
clean_model = logistic_regression_train(logisticRegr, X_clean_train, y_clean_train)

C:\Users\sathv\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [448]:
y_clean_pred = logistic_regression_test(logisticRegr, X_clean_test)

In [449]:
y_clean_pred

array([[0.89548417, 0.10451583],
       [0.91928258, 0.08071742],
       [0.53420644, 0.46579356],
       ...,
       [0.24316377, 0.75683623],
       [0.90430599, 0.09569401],
       [0.95870083, 0.04129917]])

In [450]:
clean_diff = average_diff(y_clean_pred, X_clean_test)

In [451]:
clean_diff

0.20409457711781387

In [452]:
clean_test_score = logisticRegr.score(X_clean_test, y_clean_test)

In [453]:
clean_test_score

0.8014739751266697

In [454]:
from tabulate import tabulate
data = [["Local outlier Factor","Impute by mode","Dirty test data","Label Encoding", "Logistic Regression", dirty_test_score, dirty_diff],
["Local outlier Factor","Impute by mode", "Clean test data", "Label Encoding", "Logistic Regression", clean_test_score, clean_diff]]
print (tabulate(data, headers=["Detection Method","Method", "Test Data", "Encoding Technique", "ML Model", "Accuracy", "Fairness"]))

Detection Method      Method          Test Data        Encoding Technique    ML Model               Accuracy    Fairness
--------------------  --------------  ---------------  --------------------  -------------------  ----------  ----------
Local outlier Factor  Impute by mode  Dirty test data  Label Encoding        Logistic Regression    0.799785    0.207288
Local outlier Factor  Impute by mode  Clean test data  Label Encoding        Logistic Regression    0.801474    0.204095
